In [1]:

import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv



pn.extension()

# Connect to bank data and clean up your transactions
df = pd.read_csv('bankstatement.csv')

# Clean df
df = df[['Date', 'Description', 'Amount']] # Keep only desired columns
df = df.rename(columns={'Completed Date': 'Date'})   # Rename columns
df['Category'] = 'unassigned'    # Add category column

# Assign transactions to the correct category
df['Category'] = np.where(df['Description'].str.contains('Rent'), 'Rent', df['Category'])
df['Category'] = np.where(df['Description'].str.contains('Food'), 'Food', df['Category'])
df['Category'] = np.where(df['Description'].str.contains('Grocery'), 'Grocery', df['Category'])
df['Category'] = np.where(df['Description'].str.contains('Shopping'), 'Shopping', df['Category'])
df['Category'] = np.where(df['Description'].str.contains('Transfer'), 'Transfer', df['Category'])
df['Category'] = np.where(df['Description'].str.contains('Misc'), 'Misc', df['Category'])
df['Category'] = np.where(df['Description'].str.contains('Payroll'), 'Excluded', df['Category'])

# Convert the "Date" column to a datetime format
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# check unassigned transactions and confirm all transactions are assigned to a category
unassigned = df.loc[df['Category'] == 'unassigned']

# Create Top Banner for a summary of last month's income, recurring expenses, non-recurring expenses and savings
latest_month = df['Month'].max()
latest_year = df['Year'].max()

# Filter the dataframe to include only transactions from the latest month
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]

last_month_expenses = last_month_expenses.groupby('Category')['Amount'].sum().reset_index()

last_month_expenses['Amount'] = last_month_expenses['Amount'].astype('str')
last_month_expenses['Amount'] = last_month_expenses['Amount'].str.replace('-', '')
last_month_expenses['Amount'] = last_month_expenses['Amount'].astype('float')  # get absolute figures

last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Excluded|unassigned") == False]  # exclude "excluded" category
last_month_expenses = last_month_expenses.sort_values(by='Amount', ascending=False)  # sort values
last_month_expenses['Amount'] = last_month_expenses['Amount'].round().astype(int)  # round values

last_month_expenses_tot = last_month_expenses['Amount'].sum()

def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

# Set the default values for widgets
income_widget = pn.widgets.TextInput(name="Income", value="1002")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="510")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value="251")
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="241")

# Watch widget values for changes
income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")

# Create a bar chart for last month expenses
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='Amount', 
    height=250, 
    width=850, 
    title="Last Month Expenses",
    ylim=(0, 500),
    color='blue'  # Change bar color
)

# Prepare data for monthly expenses trend by category chart
df['Date'] = pd.to_datetime(df['Date'])            
df['Month-Year'] = df['Date'].dt.to_period('M')    
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].astype('str')
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].str.replace('-', '')
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("Excluded") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='Amount', ascending=False)
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'Amount': 'Amount '})

monthly_expenses_chart_title = "Monthly Expenses by Category"

# Define Panel widget for category selection
select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Rent',
    'Food',
    'Grocery',
    'Shopping',
    'Transfer',
    'Misc',
])

# Define plot_expenses function
def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Amount ')
    plot.opts(fontsize={'title': '12pt'}, title="Monthly Expenses by Category", title_format='center')  # Set title properties
    return plot

# Define callback function
@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# Create layout for monthly expenses by category chart
monthly_expenses_trend_by_cat_chart = pn.Row(
    select_category1,
    update_plot,
    background='#f0f0f0',
    title=monthly_expenses_chart_title  # Set the title
)

# Define summary table
df = df[['Date', 'Category', 'Amount']]
df['Amount'] = df['Amount'].astype('str')
df['Amount'] = df['Amount'].str.replace('-', '')
df['Amount'] = df['Amount'].astype('float')        
df = df[df["Category"].str.contains("Excluded") == False]    
df['Amount'] = df['Amount'].round().astype(int)      

def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]

summary_table = pn.widgets.DataFrame(filter_df('All'), height=300, width=400)

def update_summary_table(event):
    summary_table.value = filter_df(event.new)

select_category1.param.watch(update_summary_table, 'value')

# Define dropdown menu to select the month
months = ['June', 'July', 'August', 'September', 'October', 'November', 'December']
select_month = pn.widgets.Select(name='Select Month', options=months)

def filter_expenses(month, category):
    if category == 'All':
        filtered_df = df[df['Month-Year'] == f'{month}-2022']
    else:
        filtered_df = df[(df['Month-Year'] == f'{month}-2022') & (df['Category'] == category)]
    return filtered_df

def plot_monthly_expenses(month):
    month_index = months.index(month) + 6  
    filtered_df = df[df['Date'].dt.month == month_index]
    filtered_df = filtered_df[filtered_df['Category'].isin(['Rent', 'Food', 'Grocery', 'Shopping', 'Transfer', 'Misc'])]
    monthly_expenses_grouped = filtered_df.groupby('Category')['Amount'].sum()
    monthly_expenses_chart = monthly_expenses_grouped.hvplot.bar(
        height=400,
        width=800,
        title=f"Monthly Expenses for {month}",
        xlabel="Category",
        ylabel="Total Expenses",
        color='orange'  
    )
    return monthly_expenses_chart

@pn.depends(select_month.param.value)
def update_monthly_expenses_chart(month):
    return plot_monthly_expenses(month)

monthly_expenses_dropdown_chart = pn.Column(
    update_monthly_expenses_chart
)

# Update main layout with styling
# Update main layout with styling
template = pn.template.FastListTemplate(
    title="Monthly Expenses Summary",
    header_background='#006400',  # Dark green color for the top bar
    sidebar=[
        pn.pane.PNG('https://www.personalfinanceplan.in/wp-content/uploads/2022/05/Personal-Finance-Plan_Hero-Image-01.jpg', sizing_mode='scale_both'),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        select_month,
        select_category1
    ],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950, align='center', styles={'background': '#f0f0f0'}),
        pn.Row(last_month_expenses_chart, align='center', styles={'background': '#f0f0f0'}),
        pn.Row(monthly_expenses_dropdown_chart, align='center', styles={'background': '#f0f0f0'}),
        pn.GridBox(
            monthly_expenses_trend_by_cat_chart[1],
            summary_table,
            ncols=2,
            min_height=300,
            align='start',
            styles={'background': '#f0f0f0'},
        )
    ]
)

template.show()



C:\Users\haris\AppData\Local\Temp\ipykernel_5112\3678899581.py:131: PanelDeprecationWarning: "Row(..., background='#f0f0f0')" is deprecated and will be removed in version 1.3, use "Row(..., styles={'background': '#f0f0f0'})" instead.
  monthly_expenses_trend_by_cat_chart = pn.Row(


Launching server at http://localhost:59045
